In [1]:
#since noteboke doesn't work in jupiterlabs    %matplotlib notebook 
%matplotlib notebook
%load_ext autoreload
%autoreload 2

<h1> All code in this repo optimized for my linux computer. change the os.chdir path in all files to be dynamic befre posting code! </h1>

In [4]:
import os
os.chdir('/home/dana_z/ssd_2TB/6OHDA')
#import mpld3
#mpld3.enable_notebook()
import numpy as np
import scipy as sci
from scipy import signal
from matplotlib import pyplot as plt
from matplotlib import gridspec
import matplotlib.colors as Mcolors
import matplotlib.cm as cmx
import sys
import h5py
from IO import *
from utils import *
from plotUtils import *
from ColorSchems import colorPallet as CP
import pptx
from pptx import Presentation 
from pptx.util import Inches
from io import BytesIO
import re
import warnings
import pandas as pd
import sqlalchemy as db
import gc
from tqdm import tqdm
import seaborn as sns
import pywt # wavelet package

FileNotFoundError: [WinError 3] The system cannot find the path specified: '/home/dana_z/ssd_2TB/6OHDA'

In [5]:
Files = ['FinalData_6OHDA.h5','FinalData_6OHDA_H.h5','FinalData_6OHDA_H_skip.h5','FinalData_6OHDA_skip.h5']
miceList = getMiceList(Files[0])
# data for Beta Mice - NOTICE: ONLY 1222 is CHI rest are PV
betaMice = ['8430','4539','7584','7909','1222']

In [ ]:
# data for all mice - homomorphic filter - Only Amphetamine sessions
HfPVdataH = getData(Files[1],['speed','trace'],period ='Pre', cre='PV',red = True,day = lambda x: x==0,drug=b'Amphetamin')
HfPVdataP = getData(Files[1],['speed','trace'],period ='Pre', cre='PV',red = True,day = lambda x: x>0,drug=b'Amphetamin')
HfMSNdataH = getData(Files[1],['speed','trace'],period ='Pre',red = False,day = lambda x: x==0,drug=b'Amphetamin')
HfMSNdataP = getData(Files[1],['speed','trace'],period ='Pre',red = False,day = lambda x: x>0,drug=b'Amphetamin')
HfCHIdataH = getData(Files[1],['speed','trace'],period ='Pre', cre='CHI',red = True,day = lambda x: x==0,drug=b'Amphetamin')
HfCHIdataP = getData(Files[1],['speed','trace'],period ='Pre', cre='CHI',red = True,day = lambda x: x>0,drug=b'Amphetamin')


In [ ]:
def periodCalc(day):
    if day== 0:
        return 'Healthy'
    elif day<5:
        return 'Day 1-4'
    elif day<13:
        return 'Day 5-12'
    elif day<21:
        return 'Day 13-20'
    else:
        return 'One Month'

Get pre and post firing rates into a df 

In [ ]:
#f = h5py.File('OnsetsAndPeriods.hdf5','r')
col_names = ['Mouse','Sess','Day','Period','drugPeriod','start_high',
             'start_med','start_low','All_','end_high','end_med',
             'end_low','acc','dcc']
df = pd.DataFrame(columns = col_names)
cond = ['PVdataH','MSNdataH','CHIdataH','PVdataP','MSNdataP','CHIdataP']
transWindow = np.ones((10,1))
for c in cond:
    data = eval(c)
    for sess in data.keys():
        try:
            if sess[5] == 'B':
                day = 0
            else:
                day = int(re.findall(r'\d+',sess[5:])[0])
            m = sess[0:4]
            period = periodCalc(day)
            
            for drugPeriod in ['Pre','Post']:

                Hp = getOnsetOrPeriod(m,sess,drugPeriod,'HighMvmtPeriods_vector')
                Lp = getOnsetOrPeriod(m,sess,drugPeriod,'LowMvmtPeriods_vector')
                Mp = ((Hp+Lp)==0)
                Ca = getOnsetOrPeriod(m,sess,drugPeriod,'caOnset_Hf')
                CaO = Ca[:,1:]-Ca[:,:-1]
                if CaO.shape[1]<Hp.shape[0]:
                     CaO = np.append(CaO,np.zeros((CaO.shape[0],1)),axis=1)
                C_start = (CaO==1)*1
                C_end = (CaO==-1)*1
    #            Acc = np.append(np.convolve(np.diff(Lp*1+Mp*2+Hp*3)>0, transWindow[:,0], mode='same')>0,False)
    #            Dcc = np.append(np.convolve(np.diff(Lp*1+Mp*2+Hp*3)<0, transWindow[:,0], mode='same')>0,False)
                Acc = np.append(np.convolve(np.diff(Lp*1+Hp*2)>0, transWindow[:,0], mode='same')>0,False)
                Dcc = np.append(np.convolve(np.diff(Lp*1+Hp*2)<0, transWindow[:,0], mode='same')>0,False)
                a = np.asarray([np.sum(C_start[:,Hp],axis=1),np.sum(C_start[:,Mp],axis=1),
                                np.sum(C_start[:,Lp],axis=1),np.sum(C_start,axis=1),
                                np.sum(C_end[:,Hp],axis=1),np.sum(C_end[:,Mp],axis=1),np.sum(C_end[:,Lp],axis=1),
                                np.sum(C_end[:,Acc],axis=1),np.sum(C_end[:,Dcc],axis=1)]).T
                df2 = pd.DataFrame(a, columns = col_names[3:])
                df2['totalTime'] = (Hp.shape[0]/1200)
                df2['lowTime'] = np.sum(Lp/1200)
                df2['medTime'] = np.sum(Mp/1200)
                df2['highTime'] = np.sum(Hp/1200)
    #            df2['accTrans'] = np.sum(np.diff(Lp*1+Mp*2+Hp*3)>0)
    #            df2['dccTrans'] = np.sum(np.diff(Lp*1+Mp*2+Hp*3)<0)
                df2['accTrans'] = np.sum(np.diff(Lp*1+Hp*2)>0)
                df2['dccTrans'] = np.sum(np.diff(Lp*1+Hp*2)<0)
                df2['Day'] = day
                df2['Mouse'] = m
                df2['Sess'] = sess
                df2['CellType'] = c[:-5]
                df2['drugPeriod'] = drugPeriod
                df2['Beta'] = (m in betaMice) 
                df = pd.concat([df,df2],sort=True)
        except:
            print(sess)



In [ ]:
df['High'] =  df.apply (lambda row: row.start_high/row.highTime if row.highTime != 0 else row.start_high, axis=1)
df['Medium'] = df.apply (lambda row: row.start_med/row.medTime, axis=1)
df['Low'] = df.apply (lambda row: row.start_low/row.lowTime , axis=1)
df['All'] = df.apply (lambda row: row.All_/row.totalTime , axis=1)
df['Acc'] = df.apply (lambda row: row.acc/row.accTrans , axis=1)
df['Dcc'] = df.apply (lambda row: row.dcc/row.dccTrans , axis=1)
df['Acc_'] = df.apply (lambda row: row.acc/row.All_ if row.All_ >0 else 0 , axis=1)
df['Dcc_'] = df.apply (lambda row: row.dcc/row.All_ if row.All_ >0 else 0 , axis=1)

df.to_csv('D1D2_Analysis')
df.head()